In [6]:
import json

from base64 import urlsafe_b64decode, urlsafe_b64encode
from datetime import datetime
from hashlib import sha256
from math import ceil
from typing import Tuple
import msgpack

CONTEXTS = [
    "https://www.w3.org/2018/credentials/v1",
    "https://w3id.org/security/data-integrity/v2",
    { "@vocab": "https://www.w3.org/ns/credentials/issuer-dependent#" }
  ]

input = json.load(open("v2JSON/presentation.json", "r"))

In [7]:
def base64_encode(val: bytes) -> str:
    return urlsafe_b64encode(val).rstrip(b"=").decode("utf-8")

def base64_decode(val: str) -> bytes:
    padlen = 4 - len(val) % 4
    return urlsafe_b64decode(val if padlen > 2 else (val + "=" * padlen))

def encode_bytes(val: bytes) -> bytes:
    return len(val).to_bytes(2, "big") + val

def encode_identifier(ident: str) -> str:
    return "did:sov:" + ident.replace(" ", "%20")

def decode_identifier(ident: str) -> str:
    return ident.replace("%20", " ").lstrip("did:sov:")

# V2

In [8]:
def encode_proofs(proofs, context):
    return 'u' + base64_encode(msgpack.dumps([context, proofs]))
def decode_proofs(proofs):
    return msgpack.loads(base64_decode(proofs[1:]))

In [62]:
def extract_related_statements(statements):
    sig_to_related = {}
    sig_def = {}

    # Identify all "Signature" statements
    for st_id, st_data in statements.items():
        if "Signature" in st_data and \
           "issuer" in st_data["Signature"] and \
           "id" in st_data["Signature"]["issuer"] and \
           "schema" in st_data["Signature"]["issuer"] and \
           "id" in st_data["Signature"]["issuer"]["schema"]:
            sig_to_related[st_id] = [st_id]
            sig_def[st_id] = {
                "issuer": {'id': st_data["Signature"]["issuer"]["id"],
                           'schema': st_data["Signature"]["issuer"]["schema"]["id"]},
            }

    # Relate other statements to these "Signature" statements
    for st_id, st_data in statements.items():
        ref_id = st_data.get("Revocation", {}).get("reference_id") or \
                 st_data.get("VerifiableEncryption", {}).get("reference_id") or \
                 st_data.get("Membership", {}).get("reference_id") or \
                 st_data.get("Commitment", {}).get("reference_id") or \
                 st_data.get("Range", {}).get("signature_id")
        if ref_id and ref_id in sig_to_related:
            sig_to_related[ref_id].append(st_id)

    return sig_to_related, sig_def

related_groups_updated, sig_def = extract_related_statements(input['schema']['statements'])
print(sig_def)

{'48f57e057e1d8a0dd0053c9a0fb0aa40': {'issuer': {'id': 'd1bc208a7d0695e8cf2f3a3fe3e1f620', 'schema': '89c31714307f6522379ad99b473ee364'}}}


In [66]:
def extractCredential(presentation:dict)->dict:
    ret = []
    presentation_requests = presentation["schema"] # this is still required to obtain statement schema
    presentation_proofs = presentation["presentation"]['proofs']

    tmp_messages = {}
    disclosed_messages = presentation["presentation"]["disclosed_messages"]
    if disclosed_messages:
        for messages in disclosed_messages:
            tmp_messages[messages[0]] = messages[1]

    related_statements, sig_def = extract_related_statements(presentation_requests['statements'])
    for sig_id, rel_sts in related_statements.items():
        issuer_info = sig_def[sig_id]
        
        cred_subject = {}
        proofs = {}
        for i in rel_sts:
            proofs[i] = presentation_proofs[i]
            cred_subject[list(presentation_proofs[i].keys())[0]] = None
        
        #TODO verify the statements first, build credential subject
        
        cred_proofValue = encode_proofs(proofs, 2)
        cred = {
            "@context": CONTEXTS.copy(),
            "type": ["VerifiableCredential", "AnonCredsPresentation"],
            "issuer": {
                "id": encode_identifier(issuer_info['issuer']['id']),
                "schema": encode_identifier(issuer_info['issuer']['schema']),
            },
            "issuanceDate": datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ"),
            "credentialSubject": cred_subject,
            "proof": {
                "type": "AnonCredsPresentationProof2022",
                "proofValue": cred_proofValue
            },
        }
        ret.append(cred)
    return ret
extractCredential(input)

[{'@context': ['https://www.w3.org/2018/credentials/v1',
   'https://w3id.org/security/data-integrity/v2',
   {'@vocab': 'https://www.w3.org/ns/credentials/issuer-dependent#'}],
  'type': ['VerifiableCredential', 'AnonCredsPresentation'],
  'issuer': {'id': 'did:sov:d1bc208a7d0695e8cf2f3a3fe3e1f620',
   'schema': 'did:sov:89c31714307f6522379ad99b473ee364'},
  'issuanceDate': '2024-02-29T04:36:03Z',
  'credentialSubject': {'Signature': None,
   'Revocation': None,
   'Commitment': None,
   'VerifiableEncryption': None,
   'Range': None,
   'Membership': None},
  'proof': {'type': 'AnonCredsPresentationProof2022',
   'proofValue': 'ukgKG2SA0OGY1N2UwNTdlMWQ4YTBkZDAwNTNjOWEwZmIwYWE0MIGpU2lnbmF0dXJlg6JpZNkgNDhmNTdlMDU3ZTFkOGEwZGQwMDUzYzlhMGZiMGFhNDCyZGlzY2xvc2VkX21lc3NhZ2VzgaEx2UAzNWU0YmQ0MzAzYTgwZjFjZTY1ZGFhY2FkY2M2ZDQ3ZTgwNDFiNjBmM2VhODEwYWFlMmRjYmFjYTIzZDExYThio3Bva4Snc2lnbWFfMdlgYTMzYzIyZTFkNTJiYTgwY2QyMTlhZDQ2NGFkYWE4MDhiMjE1NGM5MGU3OTQ1YTMwYmM4NTMwMzQzMzRmMjBhMjlhZjdhNmU3M2VjY2JlOGE3Z

In [68]:
def to_w3c_presentation(presentation:dict)->dict:

    credentials = extractCredential(presentation)

    result = {
        "@context": CONTEXTS,
        "type": ["VerifiablePresentation"],
        "schema": encode_identifier(presentation["schema"]["id"]), # this is still left as presentation request id
        "verifiableCredential": credentials, # May or may not exist
        "proof": {
            "cryptosuite": "anoncreds-2024",
            "type": "DataIntegrityProof",
            "proofPurpose": "authentication",
            "verificationMethod": "did:key:z6MkwXG2WjeQnNxSoynSGYU8V9j3QzP3JSqhdmkHc6SaVWoT/credential-definition",
            "proofValue": encode_proofs(presentation["presentation"]["challenge"],3) # challenge
        }

    }
    return result
tmp = to_w3c_presentation(input)
tmp

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://w3id.org/security/data-integrity/v2',
  {'@vocab': 'https://www.w3.org/ns/credentials/issuer-dependent#'}],
 'type': ['VerifiablePresentation'],
 'schema': 'did:sov:9df284e24fb08e4869317177961e4e64',
 'verifiableCredential': [{'@context': ['https://www.w3.org/2018/credentials/v1',
    'https://w3id.org/security/data-integrity/v2',
    {'@vocab': 'https://www.w3.org/ns/credentials/issuer-dependent#'}],
   'type': ['VerifiableCredential', 'AnonCredsPresentation'],
   'issuer': {'id': 'did:sov:d1bc208a7d0695e8cf2f3a3fe3e1f620',
    'schema': 'did:sov:89c31714307f6522379ad99b473ee364'},
   'issuanceDate': '2024-02-29T04:41:09Z',
   'credentialSubject': {'Signature': None,
    'Revocation': None,
    'Commitment': None,
    'VerifiableEncryption': None,
    'Range': None,
    'Membership': None},
   'proof': {'type': 'AnonCredsPresentationProof2022',
    'proofValue': 'ukgKG2SA0OGY1N2UwNTdlMWQ4YTBkZDAwNTNjOWEwZmIwYWE0MIGpU2l

## Decode

In [73]:
def to_AnonCreds_presentation(presentation:dict) -> dict:
    
    presentation_schema_id = decode_identifier(presentation["schema"])
    statements = {}
    proofs = {}
    disclosed_messages = []
    
    # reconstruct statements and proofs from VC
    for cred in presentation['verifiableCredential']:
        tmp_proof = decode_proofs(cred['proof']['proofValue'])[1]
        proofs = {**proofs, **tmp_proof}

        #TODO reconstruct disclosed messages from credential subject

    template = {
        "schema": {
            "id":presentation_schema_id # statements can no longer be constructed
        },
        "presentation": {
            "proofs": proofs,
            "challenge":decode_proofs(presentation['proof']['proofValue'])[1],
            "disclosed_messages": disclosed_messages
        }
    }
    
    return template
to_AnonCreds_presentation(tmp)

{'schema': {'id': '9df284e24fb08e4869317177961e4e64'},
 'presentation': {'proofs': {'48f57e057e1d8a0dd0053c9a0fb0aa40': {'Signature': {'id': '48f57e057e1d8a0dd0053c9a0fb0aa40',
     'disclosed_messages': {'1': '35e4bd4303a80f1ce65daacadcc6d47e8041b60f3ea810aae2dcbaca23d11a8b'},
     'pok': {'sigma_1': 'a33c22e1d52ba80cd219ad464adaa808b2154c90e7945a30bc853034334f20a29af7a6e73eccbe8a7f2258bc8d03200a',
      'sigma_2': 'a78a5599edc390214c2a20a7c0ed3b2dbb9819d6e33141ac5dc42603a1173b6ede969cf975acded9eb084f7302232382',
      'commitment': 'a68c61f4aaf9b0e4726b2ee6eea03ab46a354980cd692fb079d78f273a7495621b3317a7498d12e330d0c32335c022ec0d5fb610899a8cea93a73455fb5b18861cf7a2c3edce83562442da10e277ff5a40c2f38471add3d793d25fcec72900ce',
      'proof': ['43c8cafc3a52caa9fba011a5d22aa6239dbdd0613dd13d9eff6094773b1d36fc',
       '5deeaa88b266862b663da31f153a7ee127a87214907823fe9a62f708ef24ff1e',
       '73129f1b580fba8d917a7134446eee276974e4b5de8f855ebb53ba061aff6273',
       '2eb368505a2306504e4ddb

# V1

In [4]:
# def encodeProofs(decoded):
#     return base64_encode(json.dumps(decoded).encode('utf-8'))
# def decodeProofs(encoded):
#     return base64_decode(encoded).decode('utf-8')

In [5]:
# def extractCredential(presentation:dict)->dict:
#     ret = []
#     presentation_st = presentation["schema"]["statements"]

#     tmp_messages = {}
#     disclosed_messages = presentation["presentation"]["disclosed_messages"]
#     if disclosed_messages:
#         for messages in disclosed_messages:
#             tmp_messages[messages[0]] = messages[1]

#     # Check if a issuer exist in the credential ow error
#     for st in presentation_st.keys():
#         st_type = list(presentation_st[st].keys())[0]
#         if st_type == "Signature":
#             issuer = presentation_st[st][st_type]["issuer"]['id']
#             cred_def_id = presentation_st[st][st_type]["issuer"]["schema"]["id"]
#             break
#     if issuer == "" or cred_def_id == "":
#         raise Exception("No issuer or credential definition id found in the credential")

#     for idx, st in enumerate(presentation_st.keys()):
#         st_type = list(presentation_st[st].keys())[0]
#         rest = presentation_st[st]
#         try: 
#             tmp = presentation["presentation"]["proofs"][st]
#             if list(tmp.keys())[0] != st_type:
#                 raise Exception("Proof type mismatched for the statement")
#             proof = encodeProofs({"schema": rest, "proof": tmp})
#         except:
#             raise Exception("No proof found for the statement")
        
#         credential_subject = {
#             "type": st_type + " Statement",
#         }
#         # need improvement
#         if st in tmp_messages.keys():
#             value = list(tmp_messages[st][0][1].values())[0]
#             if value["print_friendly"]:
#                 credential_subject[tmp_messages[st][0][0]] = value["value"]

#         cred = {
#             "@context": CONTEXTS.copy(),
#             "type": ["VerifiableCredential", "AnonCredsPresentation"],
#             "issuer": {
#                 "id": encode_identifier(issuer),
#             },
#             "issuanceDate": datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ"),
#             "credentialSubject": credential_subject,
#             "proof": {
#                 "type": "AnonCredsPresentationProof2022",
#                 "proofValue": proof
#             },
#         }
#         ret.append(cred) 
#     return ret

In [6]:
# def to_w3c_presentation(presentation:dict)->dict:

#     credentials = extractCredential(presentation)
#     proofValue = encodeProofs(presentation["schema"])

#     result = {
#         "@context": CONTEXTS,
#         "type": ["VerifiablePresentation","AnonCredsPresentation"],
#         "schema": encode_identifier(presentation["schema"]["id"]),
#         "verifiableCredential": credentials, # May or may not exist
#         "proof": {
#             "type": "AnonCredsPresentationProof2022",
#             "created": datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ"),
#             "proofPurpose": "authentication",
#             "challenge":presentation["presentation"]["challenge"],
#             "proofValue": proofValue
#         }

#     }
#     return result

In [7]:
# def to_AnonCreds_presentation(presentation:dict) -> dict:
#     presentation_schema_id = decode_identifier(presentation["schema"])
#     statements = {}
#     proofs = {}
#     disclosed_messages = []
    
#     # reconstruct statements and proofs from VC
#     for cred in presentation['verifiableCredential']:
#         # statement_id = decode_identifier(cred["credentialSubject"]['id'])
#         statment_type = cred["credentialSubject"]['type'].split(" ")[0]

#         tmp_statement = json.loads(decodeProofs(cred['proof']['proofValue']))["schema"]
#         tmp_proof = json.loads(decodeProofs(cred['proof']['proofValue']))["proof"]
#         statement_id = tmp_proof[statment_type]["id"]

#         statements[statement_id] = tmp_statement
#         proofs[statement_id] = tmp_proof

#         tmp_subject = cred["credentialSubject"].copy()
#         tmp_subject.pop("type")
#         if tmp_subject:
#             tmp_mes = []
#             for k,v in tmp_subject.items():
#                 if type(v) == int:
#                     tmp_mes.append([k,{"Number":{"value": v, "print_friendly": True}}])
#                 elif type(v) == str:
#                     tmp_mes.append([k, {"Hashed":{"value": v, "print_friendly": True}}])
#                 else:
#                     raise Exception("Unkown type of value")
#             disclosed_messages.append([statement_id, tmp_mes])

#     template = {
#         "schema": {
#             "id":presentation_schema_id,
#             "statements":statements
#         },
#         "presentation": {
#             "proofs": proofs,
#             "challenge":presentation['proof']['challenge'],
#             "disclosed_messages": disclosed_messages
#         }
#     }
    
#     return template

In [8]:
# output = to_w3c_presentation(input)
# output

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://andrewwhitehead.github.io/anoncreds-w3c-mapping/schema.json',
  {'@vocab': 'urn:anoncreds:attributes#'}],
 'type': ['VerifiablePresentation', 'AnonCredsPresentation'],
 'schema': 'did:sov:9df284e24fb08e4869317177961e4e64',
 'verifiableCredential': [{'@context': ['https://www.w3.org/2018/credentials/v1',
    'https://andrewwhitehead.github.io/anoncreds-w3c-mapping/schema.json',
    {'@vocab': 'urn:anoncreds:attributes#'}],
   'type': ['VerifiableCredential', 'AnonCredsPresentation'],
   'issuer': {'id': 'did:sov:d1bc208a7d0695e8cf2f3a3fe3e1f620'},
   'issuanceDate': '2024-02-20T19:14:49Z',
   'credentialSubject': {'type': 'Signature Statement', 'name': 'John Doe'},
   'proof': {'type': 'AnonCredsPresentationProof2022',
    'proofValue': 'eyJzY2hlbWEiOiB7IlNpZ25hdHVyZSI6IHsiZGlzY2xvc2VkIjogWyJuYW1lIl0sICJpZCI6ICI0OGY1N2UwNTdlMWQ4YTBkZDAwNTNjOWEwZmIwYWE0MCIsICJpc3N1ZXIiOiB7ImlkIjogImQxYmMyMDhhN2QwNjk1ZThjZjJmM2EzZmUzZTFmNj